In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
from torch.utils.data import TensorDataset
import torch.optim as optim
from torch.utils.data import DataLoader
import cv2 
from torch.optim.lr_scheduler import StepLR
import json
from torch.utils.data import DataLoader, random_split
from utils import * 
import math
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch.autograd import Variable

In [2]:
CHARS = [
         '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
         'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K',
         'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
         'W', 'X', 'Y', 'Z', 'I', 'O', '-'
         ]

In [3]:
# Instantiate the network with 6 stages
CPMmodel = CPMLicensePlateNet(num_stages=6)
CPMmodel.load_state_dict(torch.load("./cpmweight_11.pth"))
CPMmodel.to('cuda').eval()

CPMLicensePlateNet(
  (CPMStage_x_1): CPMStage_x(
    (conv1): Conv2d(3, 32, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(32, 32, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (conv4): Conv2d(32, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu4): ReLU(inplace=True)
    (conv5): Conv2d(64, 32, kernel_size=(5, 5), stride=

In [4]:
test_img = cv2.imread("./761TJL.jpg")

test_img_resize = cv2.resize(test_img,(100,100))
original_img = test_img_resize.copy()
img = original_img/255.0
img = np.expand_dims(img, axis=0)
img = torch.from_numpy(img).float()
img = img.permute(0, 3, 1, 2)
img = img.to('cuda:0')
print(img.shape)
s1_g1_out,  s2_g2_out,  s3_g2_out,  s4_g2_out,  s5_g2_out,  s6_g2_out,  s7_g2_out2 = CPMmodel(img)
output_heatmap = s7_g2_out2[0]
output_heatmap = output_heatmap.unsqueeze(0)  # x的形状变为[1, 4, 50, 50]
output_heatmap = F.interpolate(output_heatmap, size=(100, 100), mode='bilinear', align_corners=False)
output_heatmap = output_heatmap.squeeze(0)  # 移除增加的维度，形状变回[4, 100, 100]
confirm_point=[]
for i in range(4):
    y=np.argmax(output_heatmap[i].to('cpu').detach().numpy())//100
    x=np.argmax(output_heatmap[i].to('cpu').detach().numpy())%100
    confirm_point.append((x,y))
print(confirm_point)

torch.Size([1, 3, 100, 100])
[(7, 28), (94, 36), (94, 76), (8, 69)]


In [5]:
#假设 cropped_image_2 已经是一个加载的图像
image = test_img_resize.copy()
#使用 confirm_point 作为原始点坐标
pts1 = np.float32(confirm_point)
# 目标点坐标
pts2 = np.float32([[0, 0], [100, 0], [100, 50], [0, 50]])

# 计算透视变换矩阵
matrix = cv2.getPerspectiveTransform(pts1, pts2)

# 应用透视变换
CPM_result = cv2.warpPerspective(image, matrix, (100, 50))
CPM_result = cv2.resize(CPM_result,(94,48))

# img_pil = Image.fromarray(CPM_result)
# img_pil.show()

In [6]:
from data.load_data import CHARS, CHARS_DICT, LPRDataLoader
import time

#LPRmodel = build_lprnet(lpr_max_len=8, phase=False, class_num=len(CHARS), dropout_rate=0)
LPRmodel = LPRNet(lpr_max_len=7, phase=False, class_num=len(CHARS), dropout_rate=0.5)
LPRmodel.load_state_dict(torch.load("./LPRaddbatch/best_ACC_iteration_70299.pth"))

LPRmodel.to("cuda")
print("Successful to build network!")

Successful to build network!


In [7]:
Image = CPM_result
Image = Image.astype('float32')
Image -= 127.5
Image *= 0.0078125
Image = np.transpose(Image, (2, 0, 1))

torchimgs = torch.from_numpy(Image)
torchimgs = torch.unsqueeze(torchimgs, 0)
torchimgs = torchimgs.cuda()

# forward
LPRmodel.eval()
prebs = LPRmodel(torchimgs)
# greedy decode
prebs = prebs.cpu().detach().numpy()

preb = prebs[0]
preb_label = []
no_repeat_blank_label = []

for j in range(18):
    preb_label.append(np.argmax(preb[:, j], axis=0))

pre_c = preb_label[0]
if pre_c != len(CHARS) - 1:
    no_repeat_blank_label.append(pre_c)

for c in preb_label:
    if (pre_c == c) or (c == len(CHARS) - 1):
        if c == len(CHARS) - 1:
            pre_c = c
        continue
    no_repeat_blank_label.append(c)
    pre_c = c


lb = ""
for i in no_repeat_blank_label:
    lb += CHARS[i]

print("predict: ", lb)


predict:  761TJL
